In [1]:
import censusdata

In [5]:
import pandas as pd

In [7]:
import requests
from bs4 import BeautifulSoup

In [2]:
# set variables for census data
api_key='7502bfbf0a9ccaeb30b91a8e2c0e9425e216c8b5'
dsource='zbp'
state='53'
zipcode='98125, 98115,98105,98195,98102,98101,98104,98134,98144,98108,98118,98188'

In [9]:
# 2019: pull median income in the past 12 months (in inflation-adjusted dollars) by place of birth
data_url = f'https://api.census.gov/data/2019/acs/acs5/subject?get=NAME,S1903_C01_001E&for=zip%20code%20tabulation%20area:{zipcode}&key=7502bfbf0a9ccaeb30b91a8e2c0e9425e216c8b5'
income_response=requests.get(data_url)
print(income_response.text)

# pull 2019 median income into dataframe
data=income_response.json()
medianIncome=pd.DataFrame(data[1:], columns=data[0])
medianIncome.head()

# add year column to 2019 data
medianIncome['Year'] = '2019'
medianIncome

# 2011-2019: pull median income in the past 12 months (in inflation-adjusted dollars) by place of birth
years=list(range(2011,2019))
allUrls = [f'https://api.census.gov/data/'+str(y)+"/acs/acs5/subject?get=NAME,S1903_C01_001E&for=zip%20code%20tabulation%20area:*&in=state:53&key=7502bfbf0a9ccaeb30b91a8e2c0e9425e216c8b5" for y in years]

# pull 2010-2018 median income into dataframe
year=2011
zipcodes=['98125', '98115','98105','98195','98102','98101','98104','98134','98144','98108','98118','98188']
allDataFramesIncome=[]
for url in allUrls:
    income_response=requests.get(url)
    data=income_response.json()
    medianIncome=pd.DataFrame(data[1:], columns=data[0])
    medianIncome['Year']=year
    medianIncome=medianIncome[medianIncome['zip code tabulation area'].isin(zipcodes)]
    allDataFramesIncome.append(medianIncome)
    year+=1
    
pd.concat(allDataFramesIncome, ignore_index=True)

# clean up median income data frame
medianIncome=medianIncome.rename({'S1903_C01_001E':'Household Median Income','zip code tabulation area':'Zip Code'}, axis=1)
medianIncome.drop(medianIncome.columns[[0,2]], axis=1, inplace = True)
medianIncome


[["NAME","S1903_C01_001E","zip code tabulation area"],
["ZCTA5 98104","8286","98104"],
["ZCTA5 98101","9199","98101"],
["ZCTA5 98105","17146","98105"],
["ZCTA5 98118","17318","98118"],
["ZCTA5 98134","221","98134"],
["ZCTA5 98125","19283","98125"],
["ZCTA5 98102","15650","98102"],
["ZCTA5 98108","8491","98108"],
["ZCTA5 98188","9217","98188"],
["ZCTA5 98195","0","98195"],
["ZCTA5 98115","22922","98115"],
["ZCTA5 98144","13947","98144"]]


,Household Median Income,Zip Code,Year
37,8640,98101,2018
192,16888,98118,2018
193,210,98134,2018
214,19008,98125,2018
235,15370,98102,2018
236,8645,98108,2018
244,8925,98188,2018
264,16564,98105,2018
351,22182,98115,2018
384,0,98195,2018


In [10]:

# move zip code to first column, reorder index, save to csv

# move zip code to first column
medianIncome.insert(0, 'Zip Code', medianIncome.pop('Zip Code'))
medianIncome

# reset row numbers
medianIncome.reset_index(drop=True,inplace=True)
medianIncome

# 2019: pull hispanic or latino origin by race
zipcode='98125, 98115,98105,98195,98102,98101,98104,98134,98144,98108,98118,98188'
data_url = f'https://api.census.gov/data/2019/acs/acs5?get=NAME,group(B03002)&for=zip%20code%20tabulation%20area:{zipcode}&in=state:53&key=7502bfbf0a9ccaeb30b91a8e2c0e9425e216c8b5'
race_response=requests.get(data_url)
print(race_response.text)

# pull 2019 race data into dataframe
data=race_response.json()
race_data=pd.DataFrame(data[1:], columns=data[0])
race_data.head()

# add year column to 2019 data
race_data['Year']='2019'
race_data

# 2011-2019: pull race data
allUrlsRace = [f'https://api.census.gov/data/'+str(y)+"/acs/acs5?get=NAME,group(B03002)&for=zip%20code%20tabulation%20area:*&in=state:53&key=7502bfbf0a9ccaeb30b91a8e2c0e9425e216c8b5" for y in years]

# pull 2010-2018 race data into dataframe
year=2011
zipcode=['98125', '98115','98105','98195','98102','98101','98104','98134','98144','98108','98118','98188']
allDataFrames=[]
for url in allUrlsRace:
    race_response=requests.get(url)
    data=race_response.json()
    race_data=pd.DataFrame(data[1:], columns=data[0])
    race_data['Year']=year
    race_data=race_data[race_data['zip code tabulation area'].isin(zipcode)]
    allDataFrames.append(race_data)
    year+=1

# concatenate, append
pd.concat(allDataFrames, ignore_index=True)

# append all years of data
race_data=race_data.append(allDataFrames, ignore_index=True)
race_data

# drop unwanted columns
race_data.drop(race_data.filter(regex='EA|M|NAME|state|GEO_ID|_002|01[01]|01[3-9]|02[0-1]').columns, axis=1, inplace=True)
race_data

# rename columns
race=race_data.rename({'zip code tabulation area':'Zip Code','B03002_001E': 'TotalPopulation','B03002_003E':'White alone', 'B03002_004E':'Black or African American alone', 'B03002_005E':'American Indian and Alaska Native alone', 'B03002_006E':'Asian alone', 'B03002_007E':'Native Hawaiian and Other Pacific Islander alone', 'B03002_008E':'Some other race alone', 'B03002_009E':'Two or more races alone', 'B03002_012E':'Hispanic or Latino'}, axis=1)
race

race['TotalPopulation'].apply(pd.to_numeric)

race = pd.DataFrame({
    'Year': race['Year'],
    'Zip Code': race['Zip Code'],
    'TotalPopulation': race['TotalPopulation'].apply(pd.to_numeric),
    'White alone': race['White alone'].apply(pd.to_numeric),
    'Hispanic or Latino': race['Hispanic or Latino'].apply(pd.to_numeric),
    'Black or African American alone': race['Black or African American alone'].apply(pd.to_numeric),
    'American Indian and Alaska Native alone': race['American Indian and Alaska Native alone'].apply(pd.to_numeric),
    'Asian alone': race['Asian alone'].apply(pd.to_numeric),
    'Native Hawaiian and Other Pacific Islander alone': race['Native Hawaiian and Other Pacific Islander alone'].apply(pd.to_numeric),
    'Some other race alone': race['Some other race alone'].apply(pd.to_numeric),
    'Two or more races alone': race['Two or more races alone'].apply(pd.to_numeric)
})

race = pd.DataFrame({
    'Year': race['Year'],
    'Zip Code': race['Zip Code'],
    'TotalPopulation': race['TotalPopulation'],
    'White alone': race['White alone'].div(race.TotalPopulation, axis=0),
    'Hispanic or Latino': race['Hispanic or Latino'].div(race.TotalPopulation, axis=0),
    'Black or African American alone': race['Black or African American alone'].div(race.TotalPopulation, axis=0),
    'American Indian and Alaska Native alone': race['American Indian and Alaska Native alone'].div(race.TotalPopulation, axis=0),
    'Asian alone': race['Asian alone'].div(race.TotalPopulation, axis=0),
    'Native Hawaiian and Other Pacific Islander alone': race['Native Hawaiian and Other Pacific Islander alone'].div(race.TotalPopulation, axis=0),
    'Some other race alone': race['Some other race alone'].div(race.TotalPopulation, axis=0),
    'Two or more races alone': race['Two or more races alone'].div(race.TotalPopulation, axis=0)
})
race

# move zip code to first column
race.insert(0, 'Zip Code', race.pop('Zip Code'))
race

# reset row numbers
race.reset_index(drop=True,inplace=True)
race

[["NAME","B03002_001E","B03002_001EA","B03002_001M","B03002_001MA","B03002_002E","B03002_002EA","B03002_002M","B03002_002MA","B03002_003E","B03002_003EA","B03002_003M","B03002_003MA","B03002_004E","B03002_004EA","B03002_004M","B03002_004MA","B03002_005E","B03002_005EA","B03002_005M","B03002_005MA","B03002_006E","B03002_006EA","B03002_006M","B03002_006MA","B03002_007E","B03002_007EA","B03002_007M","B03002_007MA","B03002_008E","B03002_008EA","B03002_008M","B03002_008MA","B03002_009E","B03002_009EA","B03002_009M","B03002_009MA","B03002_010E","B03002_010EA","B03002_010M","B03002_010MA","B03002_011E","B03002_011EA","B03002_011M","B03002_011MA","B03002_012E","B03002_012EA","B03002_012M","B03002_012MA","B03002_013E","B03002_013EA","B03002_013M","B03002_013MA","B03002_014E","B03002_014EA","B03002_014M","B03002_014MA","B03002_015E","B03002_015EA","B03002_015M","B03002_015MA","B03002_016E","B03002_016EA","B03002_016M","B03002_016MA","B03002_017E","B03002_017EA","B03002_017M","B03002_017MA","B030

,Zip Code,Year,TotalPopulation,White alone,Hispanic or Latino,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races alone
0,98195,2018,51,0.333333,0.372549,0.294118,0.000000,0.000000,0.000000,0.000000,0.000000
1,98101,2018,12792,0.653924,0.047608,0.059412,0.012273,0.179800,0.003049,0.001563,0.042370
2,98115,2018,53431,0.768337,0.037581,0.019801,0.000842,0.112107,0.000674,0.001703,0.058955
3,98188,2018,24935,0.354281,0.148667,0.232525,0.007620,0.158372,0.031642,0.000722,0.066172
4,98105,2018,48861,0.615092,0.050756,0.018501,0.004114,0.246495,0.002742,0.005035,0.057264
...,...,...,...,...,...,...,...,...,...,...,...
103,98125,2018,42475,0.606286,0.077787,0.091136,0.010665,0.147381,0.004167,0.002354,0.060224
104,98102,2018,25448,0.738211,0.044483,0.022124,0.002240,0.127908,0.003340,0.001611,0.060083
105,98108,2018,23980,0.275605,0.107923,0.169850,0.008173,0.369475,0.007089,0.007173,0.054712
106,98144,2018,31845,0.437840,0.118449,0.162977,0.009703,0.198618,0.001539,0.004993,0.065882


In [13]:
race

,Zip Code,Year,TotalPopulation,White alone,Hispanic or Latino,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races alone
0,98195,2018,51,0.333333,0.372549,0.294118,0.000000,0.000000,0.000000,0.000000,0.000000
1,98101,2018,12792,0.653924,0.047608,0.059412,0.012273,0.179800,0.003049,0.001563,0.042370
2,98115,2018,53431,0.768337,0.037581,0.019801,0.000842,0.112107,0.000674,0.001703,0.058955
3,98188,2018,24935,0.354281,0.148667,0.232525,0.007620,0.158372,0.031642,0.000722,0.066172
4,98105,2018,48861,0.615092,0.050756,0.018501,0.004114,0.246495,0.002742,0.005035,0.057264
...,...,...,...,...,...,...,...,...,...,...,...
103,98125,2018,42475,0.606286,0.077787,0.091136,0.010665,0.147381,0.004167,0.002354,0.060224
104,98102,2018,25448,0.738211,0.044483,0.022124,0.002240,0.127908,0.003340,0.001611,0.060083
105,98108,2018,23980,0.275605,0.107923,0.169850,0.008173,0.369475,0.007089,0.007173,0.054712
106,98144,2018,31845,0.437840,0.118449,0.162977,0.009703,0.198618,0.001539,0.004993,0.065882
